# **Funciones para Endpoints en local**

### A efectos de ayudar a la empresa en su toma de decisiones respecto a el modelo de negocio a implementar, se han creado las siguientes funciones para obtener información relevante sobre los datos de los usuarios y los viajes.

### Importamos las librerías necesarias para el desarrollo de las funciones.

In [1]:
import pandas as pd
from datetime import timedelta
from pydantic import BaseModel
import joblib

#### `Función que devuelve la predicción porcentual de oportunidad de viajes de un usuario.`

In [30]:
# Cargamos el modelo entrenado desde el archivo pickle
xgb_model = joblib.load('xgb_model.pkl')

class InputData(BaseModel):
    pBoroughID: int
    dayofweek: int
    hour: int


def predict(data: InputData):

    # Convertir los datos de entrada a un DataFrame
    input_data = pd.DataFrame([data.model_dump()])

    # Realizar la predicción
    prediction = xgb_model.predict(input_data)

    # Devolver la predicción como JSON
    return {"Probabilidad de conseguir pasajero": prediction.tolist()}

In [31]:
input_data = InputData(pBoroughID=1, dayofweek=2, hour=15)
predict(input_data)

{'Predicción de conseguir pasajero': [78.75587463378906]}

#### `Función que devuelve un top 3 de vehículos ecológicos de acuerdo al capital ingresado` 

In [13]:
def top_3_vehicles(max_price_usd: float):
    # Cargamos el DataFrame desde el archivo parquet
    vehicles_info = pd.read_parquet('Datasets/3-Normalizados/vehicles_info.parquet')  

    # Filtramos los vehículos por precio
    filtered_vehicles = vehicles_info[vehicles_info['Price (USD)'] <= max_price_usd]

    # Ordenamos los vehículos por CO2 en orden ascendente, luego por Sound Emission y finalmente por Range
    sorted_vehicles = filtered_vehicles.sort_values(by=['CO2 Emission (g/mi)', 'Sound Emission (dB)', 'Range (mi)'], ascending=[True, True, False])

    # Tomamos los tres primeros vehículos dentro del rango de precio
    top_3_vehicles = sorted_vehicles.head(3)

    # Creamos el formato de salida como una lista de diccionarios
    output_format = []
    for idx, row in enumerate(top_3_vehicles.itertuples(), start=1):
        output_format.append({
            f'Puesto {idx}': f'{row.Manufacturer} {row.Model}',
            'Precio (USD)': row._9,
            'Combustible': row.Fuel,
            'CO2': row._7,
            'dB': row._8,
            'Millas con un tanque lleno': row._10,
                
        })

    return output_format

In [14]:
top_3_vehicles(32000)

[{'Puesto 1': 'Chevrolet Bolt EV',
  'Precio (USD)': 31000,
  'Combustible': 'Electric',
  'CO2': 0,
  'dB': 67,
  'Millas con un tanque lleno': 259},
 {'Puesto 2': 'Chevrolet Equinox Hybrid',
  'Precio (USD)': 24995,
  'Combustible': 'Gasoline',
  'CO2': 25,
  'dB': 68,
  'Millas con un tanque lleno': 440},
 {'Puesto 3': 'Hyundai Tucson Hybrid',
  'Precio (USD)': 26250,
  'Combustible': 'Gasoline',
  'CO2': 25,
  'dB': 68,
  'Millas con un tanque lleno': 420}]

#### `Función que devuelve estadísticas de viajes de acuerdo al Borough ingresado`

In [2]:
def get_taxi_stats(pickup_borough: str):
   
    # Cargamos el DataFrame desde el archivo parquet
    taxis = pd.read_parquet('Datasets/3-Normalizados/taxis_2023.parquet')

    # Filtramos datos para el distrito específico
    filtered_taxis = taxis[taxis['pickup_borough'] == pickup_borough]

    # Calculamos estadísticas sobre los datos de viajes en taxi en el distrito
    total_trips = len(filtered_taxis)
        
    # Calculamos la duración promedio del viaje en horas
    duration_timedelta = (filtered_taxis['dropoff_datetime'] - filtered_taxis['pickup_datetime'])
    average_trip_duration = round(duration_timedelta.mean() / timedelta(hours=1), 2)

    # Calculamos la media de viajes por día
    daily_trips = round(filtered_taxis.groupby(filtered_taxis['pickup_datetime'].dt.date).size().mean(), 2)

    # Calculamos la media de viajes por mes
    monthly_trips = round(filtered_taxis.groupby(filtered_taxis['pickup_datetime'].dt.to_period("M")).size().mean(), 2)

    average_trip_distance = round(filtered_taxis['trip_distance'].mean(), 2)
    average_total_amount = round(filtered_taxis['total_amount'].mean(), 2)

    # Devolvemos la información estadística como JSON
    return {
        "Viajes Totales": total_trips,
        "Duración Promedio (Hs)": average_trip_duration,
        "Media de Viajes por Día": daily_trips,
        "Media de Viajes por Mes": monthly_trips,
        "Distancia recorrida promedio (millas)": average_trip_distance,
        "Total ganado en promedio (USD)": average_total_amount,
    }

In [3]:
get_taxi_stats('Manhattan')

{'Viajes Totales': 24235349,
 'Duración Promedio (Hs)': 0.25,
 'Media de Viajes por Día': 85940.95,
 'Media de Viajes por Mes': 1425608.76,
 'Distancia recorrida promedio (millas)': 3.24,
 'Total ganado en promedio (USD)': 26.35}